In [7]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [9]:
def impute_age(cols):
    age=cols[0]
    pclass=cols[1]
    if pd.isnull(age):
        if pclass==1:
            return 37
        elif pclass==2:
            return 30
        else:
            return 25
    else:
        return age

In [709]:
def fill_age(df):
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = df["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = df["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    df["Age"] = age_slice
    df["Age"] = df["Age"].astype(int)
    
    
def fill_fare(df):
    df['Rich']=pd.cut(df['Fare'],bins=[df['Fare'].min()-10,df['Fare'].mean(),df['Fare'].max()+10],labels=[0,1])
    df['Rich']=df['Rich'].astype(int)

def fill_adult(df):
    df['Adult']=pd.cut(df['Age'],bins=[df['Age'].min()-1,18,30,45,df['Age'].max()+1],labels=[0,1,2,3])
    df['Adult']=df['Adult'].astype(int)
    
    
    

In [2]:
def family_size(par,sib):
    if par+sib > 4:
        return 3
    else:
        return par+sib
    
def data_preprocessing(df):
    
    df=df.drop(['Name','Ticket','Cabin'],axis=1)
    
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

    
#     df['Age']=df['Age'].fillna(value=df['Age'].mean())

#     df['Age']=df[['Age','Pclass']].apply(impute_age,axis=1)    
    
    
    df['Fare']=df['Fare'].fillna(value=df['Fare'].mean())
    df['Fare']=df['Fare'].astype(int)
    
    fill_age(df)
    fill_fare(df)
    fill_adult(df)

    
    
    df['Embarked'] = df['Embarked'].fillna(value=df['Embarked'].mode())
    
#     df['Embarked']=df['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)
    
    df['FamilySize'] = df.apply(lambda row: family_size(row.Parch,  row.SibSp) , axis = 1)
    df['Alone']=df.apply(lambda row: int(row.Parch>0 or row.SibSp>0),axis=1)
    df=df.drop(['Parch','SibSp'],axis=1)

    
    enbarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(enbarked_one_hot)
    
    pclass_one_hot = pd.get_dummies(df['Pclass'], prefix='Pclass')
    df = df.drop('Pclass', axis=1)
    df = df.join(pclass_one_hot)
    
    adult_one_hot = pd.get_dummies(df['Adult'], prefix='Adult')
    df = df.drop('Adult', axis=1)
    df = df.join(adult_one_hot)

    return df
    

In [3]:
data=pd.read_csv('train.csv',index_col='PassengerId')
data.head()

NameError: name 'pd' is not defined

In [712]:
data=data_preprocessing(data)
data.head()

,Survived,Sex,Age,Fare,Rich,FamilySize,Alone,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Adult_0,Adult_1,Adult_2,Adult_3
PassengerId,,,,,,,,,,,,,,,,,
1,0,0,22,7,0,2,1,0,0,1,0,0,1,0,1,0,0
2,1,1,38,71,1,2,1,1,0,0,1,0,0,0,0,1,0
3,1,1,26,7,0,1,0,0,0,1,0,0,1,0,1,0,0
4,1,1,35,53,1,2,1,0,0,1,1,0,0,0,0,1,0
5,0,0,35,8,0,1,0,0,0,1,0,0,1,0,0,1,0


In [713]:
data.isnull().sum()

Survived      0
Sex           0
Age           0
Fare          0
Rich          0
FamilySize    0
Alone         0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Adult_0       0
Adult_1       0
Adult_2       0
Adult_3       0
dtype: int64

In [714]:
X = data.drop(['Survived'], axis=1).values.astype(float)


scale = StandardScaler()
X = scale.fit_transform(X)

Y = data['Survived'].values


In [715]:
# Read test data
test_df = pd.read_csv('test.csv',index_col='PassengerId')
# Prep and clean data
test_df = data_preprocessing(test_df)
# Create X_test
X_Test = test_df.values.astype(float)
# Scaling
X_Test = scale.transform(X_Test)

# Predict 'Survived'
# prediction = model.predict(X_test)

In [716]:
data.columns

Index(['Survived', 'Sex', 'Age', 'Fare', 'Rich', 'FamilySize', 'Alone',
       'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2',
       'Pclass_3', 'Adult_0', 'Adult_1', 'Adult_2', 'Adult_3'],
      dtype='object')

In [780]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data['Survived'], shuffle=True,test_size=0.3)



In [1]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression
clf = DecisionTreeClassifier()
clf=clf.fit(X_train,y_train)
pred = clf.predict(X_test)
from sklearn import metrics
metrics.accuracy_score(y_test,pred)

NameError: name 'X_train' is not defined

0.7982062780269058

In [725]:
clf=clf.fit(X,Y)
pred = clf.predict(X_Test)
submission = pd.DataFrame({
    'PassengerId': test_df.index,
    'Survived': pred,
})

submission.sort_values('PassengerId', inplace=True)    
submission.to_csv('submission.csv', index=False)

In [546]:
submission.head(10)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [414]:
test_df.head()

,Pclass,Sex,Age,Fare,Rich,FamilySize,Alone,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
892,3,0,34,7.8292,0,1,0,0,1,0
893,3,1,47,7.0000,0,2,1,0,0,1
894,2,0,62,9.6875,0,1,0,0,1,0
895,3,0,27,8.6625,0,1,0,0,0,1
896,3,1,22,12.2875,0,3,1,0,0,1
